<a href="https://colab.research.google.com/github/hegelty/hate-classifier/blob/main/%ED%98%90%EC%98%A4%ED%91%9C%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]

# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv

In [ ]:
# 데이터셋 로드
train_file = "/content/drive/MyDrive/정보/korean_unsmile_dataset/unsmile_train_v1.0.tsv"
train_dataset = []
with open(train_file, 'r') as f:
  tr = csv.reader(f, delimiter='\t')
  for row in tr:
    train_dataset.append(row)

val_file = "/content/drive/MyDrive/정보/korean_unsmile_dataset/unsmile_valid_v1.0.tsv"
val_dataset = []
with open(val_file, 'r') as f:
  tr = csv.reader(f, delimiter='\t')
  for row in tr:
    val_dataset.append(row)

In [ ]:
print(len(train_dataset), len(val_dataset))
print(val_dataset[1:4])

15006 3738
[['ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가 나 안찌르면 내가 너 찌른다 했더니 애비충 냄동충 알아서기노 ^됫^ 한번 서열 잡고 그 담에 개길때마다 더 세게나가면 확실하게 짓누를수있다익이', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['여자들은 취미가 애낳는건가.. 취미를 좀 가져라', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['개슬람녀 다 필요없고 니 엄마만 있으면 된다', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0']]


In [ ]:
from transformers import AutoModelForSequenceClassification

model_checkpoint = "beomi/KcBERT-Base"  # 사용할 모델
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)  # 모델 불러오기

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-Base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)  # 토크나이져 가져오기

In [ ]:
from tqdm.notebook import tqdm
import datasets

# 데이터셋 정제
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_list([
          {
            "text": data[0],
            "label": int(data[10] or data[11]) # 혐오표현 여부(0: 험오표현, 1: 비험오표현)
          } for data in train_dataset[1:]
        ]),
    "test": datasets.Dataset.from_list([
          {
            "text": data[0],
            "label": int(data[10] or data[11])
          } for data in val_dataset[1:]
        ]),
})

print(dataset["train"][:3])

In [ ]:
def tokenize_function(examples):  # 토크나이저 함수
    return tokenizer(examples["text"], truncation=True)

# 토크나이징
tokenized_dataset = dataset.map(tokenize_function,
                                 batched=True,  # 배치
                                 num_proc=4,  # 멀티 프로세싱
                                 remove_columns=["text"])  # text 열 제거

Map (num_proc=4):   0%|          | 0/15005 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3737 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset["train"][0])

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)  # 패딩

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # 허깅페이스 로그인

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(  # 하이퍼파라미터 설정
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(  # 데이터셋과 모델을 트레이너에 넣어줌
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.358200
1000,0.284000
1500,0.197700
2000,0.163400
2500,0.095200


Step,Training Loss
500,0.358200
1000,0.284000
1500,0.197700
2000,0.163400
2500,0.095200
3000,0.078100
3500,0.038700
4000,0.030900
4500,0.014900


TrainOutput(global_step=4690, training_loss=0.1350404135962285, metrics={'train_runtime': 839.4127, 'train_samples_per_second': 89.378, 'train_steps_per_second': 5.587, 'total_flos': 1920441426407280.0, 'train_loss': 0.1350404135962285, 'epoch': 5.0})

In [ ]:
model_name = model_checkpoint.split("/")[-1]
model.push_to_hub(  # 허깅페이스에 업로드
    f"{model_name}-finetuned-hate"
)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hegelty/KcBERT-Base-finetuned-hate/commit/ed03ac55fb85860833e802d1deaa06283357a515', commit_message='Upload BertForSequenceClassification', commit_description='', oid='ed03ac55fb85860833e802d1deaa06283357a515', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub(
    f"{model_name}-finetuned-hate"
)

CommitInfo(commit_url='https://huggingface.co/hegelty/KcBERT-Base-finetuned-hate/commit/7f31556237cf771301105e29028e4b31f87c75ce', commit_message='Upload tokenizer', commit_description='', oid='7f31556237cf771301105e29028e4b31f87c75ce', pr_url=None, pr_revision=None, pr_num=None)